In [90]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_circles
from sklearn.ensemble import RandomTreesEmbedding, ExtraTreesClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import BernoulliNB

In [91]:
import pandas as pd
import os 
print(os.getcwd())
data = pd.read_csv('../stats.txt', delim_whitespace=True)
data = data.sample(frac=1).reset_index(drop=True)

array = data.values

print(data.head())
print(data.shape)
data.columns=['col1','col2','col3','col4','col5','col6','col7','col8','col9','col10','col11','col12','col13','col14','col15','col16','col17']
#data['x'],data['y'],data['z'],data['nx'],data['ny']=data['col2'].str.split('-')
z = data['col2'].str.split('-')
print(z.head())
data['col13']=data['col13'].eq("enable").mul(1)
data['col14']=data['col14'].eq("enable").mul(1)
data['col15']=data['col15'].eq("enable").mul(1)
data['col16']=data['col16'].eq("enable").mul(1)

/Users/megha/parallel/mnt/project/progress/active
   S3D-IO    50-50-100-2-2-4-1     0.00  0.00.1  0.00.2    2.14  0.15  71.19  \
0  S3D-IO     50-50-50-2-2-2-1   260.19    0.02    0.06  326.63  0.04   0.14   
1  S3D-IO    50-50-100-2-2-4-1   182.10    0.03    0.17   24.15  0.15   6.32   
2  S3D-IO  200-200-400-2-4-4-1  1127.75    1.91    1.73  974.78  5.72   6.01   
3  S3D-IO  200-200-200-2-4-4-1  1280.43    0.95    0.76  839.91  2.86   3.49   
4  S3D-IO  200-200-400-2-2-4-1   612.72    1.91    3.19  462.31  5.72  12.67   

   0.06  0.01   1048576   1  disable disable.1 disable.2 disable.3       1024  
0  0.04  0.01     65536   5   enable   disable    enable   disable  624951296  
1  0.07  0.01     65536  18  disable   disable    enable   disable  683671552  
2  0.06  0.09   5570560   2   enable    enable   disable    enable  205520896  
3  0.08  0.06  27000832   2   enable    enable    enable    enable  225443840  
4  0.04  0.10    327680  17   enable   disable   disable   disable  1

In [92]:
X = pd.DataFrame(z.tolist())
X = X.drop(6, 1)
X['cb1']=data['col11']
X['cb2']=data['col12']
#X['cb3']=data['col13']
#X['cb4']=data['col14']
#X['cb5']=data['col15']
#X['cb6']=data['col16']
X['cb7']=data['col17']



print(type(X))
print(X.head())
i=0

for _ in X.columns:
    if(i == 0):
        continue
    X.iloc[:,i].astype(int)
    print(i)
    i=i+1 

#pd.options.display.max_rows=  1500
#print(X.iloc[:,0])



<class 'pandas.core.frame.DataFrame'>
     0    1    2  3  4  5       cb1  cb2        cb7
0   50   50   50  2  2  2     65536    5  624951296
1   50   50  100  2  2  4     65536   18  683671552
2  200  200  400  2  4  4   5570560    2  205520896
3  200  200  200  2  4  4  27000832    2  225443840
4  200  200  400  2  2  4    327680   17  120586240


In [93]:
Y = data['col8']
Y=Y.values
Y = Y*100
Y = Y.astype(int)



msk = np.random.rand(len(X)) < 0.8
train_X = X[msk]
test_X = X[~msk]
train_Y = Y[msk]
test_Y = Y[~msk]
print(type(train_X))

y = train_Y
X = train_X
#X = train_X.astype(int)


print(X[0:5])
print(type(X))

print(y.shape, X.shape)
print(y[0:5])



<class 'pandas.core.frame.DataFrame'>
     0    1    2  3  4  5       cb1  cb2        cb7
0   50   50   50  2  2  2     65536    5  624951296
1   50   50  100  2  2  4     65536   18  683671552
2  200  200  400  2  4  4   5570560    2  205520896
4  200  200  400  2  2  4    327680   17  120586240
5  200  200  200  2  4  4  31457280    2  190840832
<class 'pandas.core.frame.DataFrame'>
(892,) (892, 9)
[  14  632  601 1267  451]


In [94]:
hasher = RandomTreesEmbedding(n_estimators=10, random_state=0, max_depth=3)
X_transformed = hasher.fit_transform(X)
X_transformed

<892x73 sparse matrix of type '<class 'numpy.float64'>'
	with 8920 stored elements in Compressed Sparse Row format>

In [95]:
svd = TruncatedSVD(n_components=2)
X_reduced = svd.fit_transform(X_transformed)


In [96]:
nb = BernoulliNB()
nb.fit(X_transformed, y)


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [97]:
trees = ExtraTreesClassifier(max_depth=3, n_estimators=10, random_state=0)
trees.fit(X, y)



ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=3, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [98]:
# fig = plt.figure(figsize=(9, 8))

# ax = plt.subplot(221)
# ax.scatter(X[:, 0], X[:, 1], c=y, s=50, edgecolor='k')
# ax.set_title("Original Data (2d)")
# ax.set_xticks(())
# ax.set_yticks(())

# ax = plt.subplot(222)
# ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, s=50, edgecolor='k')
# ax.set_title("Truncated SVD reduction (2d) of transformed data (%dd)" %
#              X_transformed.shape[1])
# ax.set_xticks(())
# ax.set_yticks(())

# # Plot the decision in original space. For that, we will assign a color
# # to each point in the mesh [x_min, x_max]x[y_min, y_max].
# h = .01
# x_min, x_max = X[:, 0].min() , X[:, 0].max() 
# y_min, y_max = X[:, 1].min() , X[:, 1].max() 
# #xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

In [99]:
print(trees.predict(test_X))
print(test_Y)
trees.score(test_X, test_Y, sample_weight=None)

[ 181 3604 3663  771 3604   85   13   13 1645   88 3663   13 3663   88
  873   85 4367  873 3663   88   85   88  873  717   85   13   85  873
   13  181   85  717   88  873   85   56   85 3604   85 3604   88   85
   85   85   88  873  451 3604  451  873   85   84   85   85   85  717
   85   85 3663 3604  873   85  181   85   88   88   85   84 1133   85
   85   88  451  717   85 3663   85   85  873  873   61   88   12   13
   85 1645  873   85 3604  181  451   88   85 3604  451  873  842   85
  717  873 1645  909 3663   85  909   13  873  451   85   85   85  873
  285  717   85   88   88  873  771   85   85  842  873  873   88  873
   85   88   85  771 3604   73  451  285   88   88   85   85  771  873
   85  451   13   88   12 3604  873   88  717  909   85   88  873   85
  181  451   85  451   88  181   85  717   85 3604   85   88   88   85
   13   85  873  181   13   85 4367   85  873 3663   13   12 3663   88
 3604   88   85   85  909 3604  873 1645   88   13   85   88   88   88
  181 

0.0321285140562249